In [1]:
import pandas as pd
import numpy as np
import csv
#!pip install sqlalchemy
import datetime as dt
import sqlite3

In [2]:
#Reading tnc_bases file
df_tnc_data = pd.read_csv("tnc_bases.csv")
#Reading zone_lookup file
df_zone_data = pd.read_csv("zone_lookup.csv")

In [3]:
#Reading june_trip data file
df_trip_data = pd.read_csv("fhv_tripdata_2018-06.csv",low_memory=False)
df_trip_data.head(2)

,Pickup_DateTime,DropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Dispatching_base_number,Dispatching_base_num
0,2018-06-05 05:48:34,2018-06-05 05:58:07,NaN,265.0,NaN,B01509,NaN
1,2018-06-05 05:53:35,2018-06-05 05:58:50,NaN,265.0,NaN,B01509,NaN


In [6]:
#Column Dispatching_base_num is relevant here 
#Droping would reduce the confusion 
df_trip_data.drop(['Dispatching_base_num'], axis=1).head(2)

,Pickup_DateTime,DropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Dispatching_base_number
0,2018-06-05 05:48:34,2018-06-05 05:58:07,NaN,265.0,NaN,B01509
1,2018-06-05 05:53:35,2018-06-05 05:58:50,NaN,265.0,NaN,B01509


In [7]:
#Explorator Data Analysis:
df_trip_data.shape

(21135283, 7)

In [8]:
#Spliting the tnc_Bases columns to separate the base number and and companay app information 
df_tnc_data.tnc_Bases.str.split(expand=True,).head(2)

,0,1,2,3,4,5
0,B02510,"Tri-City,",LLC(Lyft),None,None,None
1,B02844,Endor,Car,&,"Driver,",LLC(Lyft)


In [9]:
df_tnc_data.tnc_Bases.str.split(expand=True,)[0].head(2)

0    B02510
1    B02844
Name: 0, dtype: object

In [77]:
type(df_tnc_data.tnc_Bases.str.split(expand=True,)[0])

pandas.core.series.Series

In [10]:
# creating a separate column to store the company app information
df_tnc_data['App_Comp'] = df_tnc_data.tnc_Bases.str.split().str[-1]

In [11]:
#creating a separate column of Dispatching base number to join the trip data table to get the relevant information 
df_tnc_data['Dispatching_base_number']=df_tnc_data.tnc_Bases.str.split(expand=True,)[0]
df_tnc_data.head(2)

,tnc_Bases,App_Comp,Dispatching_base_number
0,"B02510 Tri-City, LLC(Lyft)",LLC(Lyft),B02510
1,"B02844 Endor Car & Driver, LLC(Lyft)",LLC(Lyft),B02844


In [12]:
df_tnc_data.App_Comp.unique()

array(['LLC(Lyft)', 'LLC(Via)', 'Juno', 'LLC(Uber)',
       'Achtzehn-NY,LLC(Uber)', 'LLC.(Uber)',
       'Einundzwanzig-NY,LLC(Uber)', 'Zwanzig-NY,LLC(Uber)',
       'Zwolf-NY,LLC(Uber)'], dtype=object)

In [13]:
#Adding a column Entity by segregating the App type from the column App_Comp
df_tnc_data['Entity'] = df_tnc_data.App_Comp.str.extract('(Lyft|Via|Uber|Juno)')

#if (df_tnc_data.App_Comp.str.contains('Lyft', regex=False)) :
#    df_tnc_data['Entity'] = 'Lyft'
#elif (df_tnc_data.App_Comp.str.contains('Via', regex=False)) :
#    df_tnc_data['Entity'] = 'Via'
#elif (df_tnc_data.App_Comp.str.contains('uber', regex=False)) :
#    df_tnc_data['Entity'] = 'Uber'
#elif (df_tnc_data.App_Comp.str.contains('Juno', regex=False)) :
#    df_tnc_data['Entity'] = 'Juno'
#else:
#    df_tnc_data['Entity'] = 'Other'
#df_tnc_data.loc[:,'Entity']=df_tnc_data.App_Comp.apply(lambda x: pd.Series(x)[pd.Series(x).str.contains('http')].values)
#df_tnc_data['Entity'] = df_tnc_data.App_Comp.apply(lambda x: print('Lyft') if (pd.Series(x)[pd.Series(x).str.contains('Lyft')]))
#;if (pd.Series(x)[pd.Series(x).str.contains('Uber')]): return('Uber'); else: return('Others'))
#df_tnc_data.loc[:,'Entity'] = df_tnc_data.App_Comp.apply(lambda(x) : if (pd.Series(x)[pd.Series(x).str.contains('Lyft')]): return('lyft'); else: return('Uber'))
df_tnc_data.head(5)

,tnc_Bases,App_Comp,Dispatching_base_number,Entity
0,"B02510 Tri-City, LLC(Lyft)",LLC(Lyft),B02510,Lyft
1,"B02844 Endor Car & Driver, LLC(Lyft)",LLC(Lyft),B02844,Lyft
2,"B02800 Flatiron Transit, LLC(Via)",LLC(Via),B02800,Via
3,"B02914 Vulcan Cars, LLC d/b/a Juno",Juno,B02914,Juno
4,"B02871 Acht-NY, LLC(Uber)",LLC(Uber),B02871,Uber


In [14]:
df_tnc_data['Entity'].unique()

array(['Lyft', 'Via', 'Juno', 'Uber'], dtype=object)

In [15]:
#creating a dictionary to fill the entity in the trip detail dataframe
df_entity=df_tnc_data.loc[:,['Dispatching_base_number','Entity']]
df_entity.head(2)

,Dispatching_base_number,Entity
0,B02510,Lyft
1,B02844,Lyft


In [16]:
Entity_dict=dict(zip(df_entity.Dispatching_base_number,df_entity.Entity))

In [17]:
Entity_dict

{'B02510': 'Lyft',
 'B02844': 'Lyft',
 'B02800': 'Via',
 'B02914': 'Juno',
 'B02871': 'Uber',
 'B02889': 'Uber',
 'B02764': 'Uber',
 'B02835': 'Uber',
 'B02884': 'Uber',
 'B02836': 'Uber',
 'B02872': 'Uber',
 'B02887': 'Uber',
 'B02878': 'Uber',
 'B02867': 'Uber',
 'B02879': 'Uber',
 'B02598': 'Uber',
 'B02870': 'Uber',
 'B02880': 'Uber',
 'B02682': 'Uber',
 'B02875': 'Uber',
 'B02883': 'Uber',
 'B02865': 'Uber',
 'B02876': 'Uber',
 'B02617': 'Uber',
 'B02866': 'Uber',
 'B02869': 'Uber',
 'B02882': 'Uber',
 'B02877': 'Uber'}

In [90]:
df_trip_data.head(2)

,Pickup_DateTime,DropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Dispatching_base_number,Dispatching_base_num
0,2018-06-05 05:48:34,2018-06-05 05:58:07,NaN,265.0,NaN,B01509,NaN
1,2018-06-05 05:53:35,2018-06-05 05:58:50,NaN,265.0,NaN,B01509,NaN


In [19]:
#Adding Entity to trip detail to do analysis Entity wise
df_trip_data['Entity']=df_trip_data['Dispatching_base_number'].map(Entity_dict)

In [20]:
df_trip_data.Entity.unique()

array([nan, 'Lyft', 'Juno', 'Via', 'Uber'], dtype=object)

In [21]:
# replacing na values in Entity with "Others" Entity 
df_trip_data["Entity"].fillna("Others", inplace = True) 

In [22]:
df_trip_data.head(2)

,Pickup_DateTime,DropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Dispatching_base_number,Dispatching_base_num,Entity
0,2018-06-05 05:48:34,2018-06-05 05:58:07,NaN,265.0,NaN,B01509,NaN,Others
1,2018-06-05 05:53:35,2018-06-05 05:58:50,NaN,265.0,NaN,B01509,NaN,Others


In [24]:
#Selecting the trip data only the PUlocationID from EWR,JFK and LGA(1,132,138) location
df_trip_airport=df_trip_data.loc[df_trip_data['PUlocationID'].isin([1,132,138])]
df_trip_airport['pickup_date']=pd.to_datetime(df_trip_airport['Pickup_DateTime'])
df_trip_airport.head(2)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Pickup_DateTime,DropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Dispatching_base_number,Dispatching_base_num,Entity,pickup_date
267,2018-06-06 14:52:08,2018-06-06 15:23:50,138.0,161.0,NaN,B02510,NaN,Lyft,2018-06-06 14:52:08
282,2018-06-06 14:52:18,2018-06-06 15:25:47,132.0,138.0,NaN,B02510,NaN,Lyft,2018-06-06 14:52:18


In [25]:
#Extracting the Days from date to create three (10-10) days group
df_trip_airport['Days'] = pd.DatetimeIndex(df_trip_airport['pickup_date']).day

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [26]:
df_trip_airport.head(2)

,Pickup_DateTime,DropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Dispatching_base_number,Dispatching_base_num,Entity,pickup_date,Days
267,2018-06-06 14:52:08,2018-06-06 15:23:50,138.0,161.0,NaN,B02510,NaN,Lyft,2018-06-06 14:52:08,6
282,2018-06-06 14:52:18,2018-06-06 15:25:47,132.0,138.0,NaN,B02510,NaN,Lyft,2018-06-06 14:52:18,6


In [28]:
#Creating a Group colummn by assuming 1-10 days as group1,11-20 as group2 and above 21 as group3
#df_trip_airport['group']=df_trip_airport.loc[(df_trip_airport['Days']<11) & (df_trip_airport['Days']>0),1]
df_trip_airport.loc[df_trip_airport.Days <11 , 'Group'] = 1 
df_trip_airport.loc[(df_trip_airport.Days <21) & (df_trip_airport.Days >10), 'Group'] = 2
df_trip_airport.loc[(df_trip_airport.Days >20), 'Group'] = 3
df_trip_airport.head(2)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,Pickup_DateTime,DropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Dispatching_base_number,Dispatching_base_num,Entity,pickup_date,Days,Group
267,2018-06-06 14:52:08,2018-06-06 15:23:50,138.0,161.0,NaN,B02510,NaN,Lyft,2018-06-06 14:52:08,6,1.0
282,2018-06-06 14:52:18,2018-06-06 15:25:47,132.0,138.0,NaN,B02510,NaN,Lyft,2018-06-06 14:52:18,6,1.0


In [141]:
df_trip_airport.loc[(df_trip_airport.Group>2)].head(2)

,Pickup_DateTime,DropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Dispatching_base_number,Dispatching_base_num,Entity,pickup_date,month,Days,Group
24171,2018-06-25 14:00:00,2018-06-25 15:43:32,132.0,9.0,NaN,B02925,NaN,Others,2018-06-25 14:00:00,6,25,3.0
24172,2018-06-25 14:15:00,2018-06-25 15:51:27,132.0,9.0,NaN,B02925,NaN,Others,2018-06-25 14:15:00,6,25,3.0


In [103]:
#Entity wise total pick ups in june 2018
df_trip_airport.groupby('Entity')['Dispatching_base_number'].count()

Entity
Lyft      167559
Others     72299
Uber      317297
Via         5060
Name: Dispatching_base_number, dtype: int64

In [30]:
#Entity wise and date range wise pick ups count
df_trip_airport.groupby(['Entity','Group'])['Dispatching_base_number'].count()

Entity  Group
Lyft    1.0       55450
        2.0       56155
        3.0       55954
Others  1.0       23784
        2.0       24387
        3.0       24128
Uber    1.0      106184
        2.0      106084
        3.0      105029
Via     1.0         906
        2.0        2261
        3.0        1893
Name: Dispatching_base_number, dtype: int64

In [18]:
df_zone_data.columns

Index(['LocationID', 'Borough', 'Zone', 'service_zone'], dtype='object')

In [17]:
df_zone_data.groupby('service_zone')['LocationID'].count().reset_index()

,service_zone,LocationID
0,Airports,2
1,Boro Zone,205
2,EWR,1
3,Yellow Zone,55


In [121]:
d='2015-01-08 22:44:09' 
date=pd.to_datetime(d).date()
print(date)


2015-01-08
